In [1]:
from data import Data
data = Data()

In [2]:
data.data_size()

Train size is (42000, 784).
Test size is (28000, 784)


In [3]:
data.data_info()


Train:
--------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 251.2 MB

Test:
--------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [71]:
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
def pd_to_tensor(data_set):
    data_set = np.array(data_set)
    return np.array([obs.reshape(28, 28, 1) for obs in data_set])

def data_preprocess(train, train_target, test):    
    train = pd_to_tensor(train)
    test = pd_to_tensor(test)
    labels = np_utils.to_categorical(np.array(train_target), 10)
    xtrain, xtest, ytrain, ytest = train_test_split(train, labels, test_size = 0.25, random_state = 1)
    return xtrain, xtest, ytrain, ytest, test


In [72]:
xtrain, xtest, ytrain, ytest, test = data_preprocess(data.train, data.train_target, data.test)
print xtrain.shape
print ytest.shape

(31500, 28, 28, 1)
(10500, 10)


In [89]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.models import Sequential
from keras import regularizers

model = Sequential()
model.add(Conv2D(filters = 16, 
                 kernel_size = 3,
                 padding = 'same',
                 activation = 'relu',
                 input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 64, 
                 kernel_size = 2,
                 padding = 'same',
                 activation = 'relu'))
model.add(MaxPooling2D(pool_size=2))
# model.add(Dense(256, activation = 'relu', input_shape = (28, 28, 1)))
model.add(Dense(512, 
                activation = 'relu', 
                kernel_regularizer=regularizers.l2(0.02)))
# model.add(Dropout(0.1))
model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 64)        4160      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dense_31 (Dense)             (None, 7, 7, 512)         33280     
_________________________________________________________________
global_average_pooling2d_16  (None, 512)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                5130      
Total para

In [90]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [91]:
from keras.callbacks import ModelCheckpoint  

epochs = 40

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(xtrain, ytrain, 
          validation_data=(xtest, ytest),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 31500 samples, validate on 10500 samples
Epoch 1/40
31500/31500 [==============================] - 56s - loss: 2.8252 - acc: 0.6343 - val_loss: 2.3785 - val_acc: 0.7162
Epoch 2/40
31500/31500 [==============================] - 55s - loss: 2.0634 - acc: 0.7854 - val_loss: 1.9605 - val_acc: 0.8327
Epoch 3/40
31500/31500 [==============================] - 56s - loss: 1.9546 - acc: 0.8141 - val_loss: 2.0143 - val_acc: 0.8072
Epoch 4/40
31500/31500 [==============================] - 56s - loss: 1.9014 - acc: 0.8242 - val_loss: 1.9359 - val_acc: 0.8270
Epoch 5/40
31500/31500 [==============================] - 57s - loss: 1.8689 - acc: 0.8338 - val_loss: 1.9077 - val_acc: 0.8330
Epoch 6/40
31500/31500 [==============================] - 58s - loss: 1.8519 - acc: 0.8371 - val_loss: 1.9336 - val_acc: 0.8295
Epoch 7/40
31500/31500 [==============================] - 56s - loss: 1.8361 - acc: 0.8397 - val_loss: 1.8401 - val_acc: 0.8546
Epoch 8/40
31500/31500 [==============================

In [81]:
model.load_weights('saved_models/weights.best.hdf5')

In [82]:
predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test]

In [84]:
data.write_submission(predictions, 'cnn_kernel_k001_a001.csv')